In [1]:
from datetime import datetime
from openai import OpenAI

In [2]:
# 讀取環境變數(OpenAI API Key)
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# 初始化 client
client = OpenAI()

In [4]:
# 定義要給 GPT 認識的 function schema
functions = [
    {
        "name": "get_current_time",
        "description": "取得目前時間(無需參數)",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    }
]

In [5]:
# 使用者提問
user_question = "請告訴我現在的時間"

In [6]:
# 第一次呼叫 GPT 讓GPT決定是否要呼叫函數
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "你是一個可以呼叫工具的助理。"},
        {"role": "user", "content": user_question}
    ],
    functions=functions,
    function_call="auto"  # 讓 GPT 自行決定是否呼叫函數
)

In [7]:
# 印出 GPT 回傳的原始 JSON
print("\n=== GPT 回傳原始 JSON ===")
print(response)


=== GPT 回傳原始 JSON ===
ChatCompletion(id='chatcmpl-BrPIx0jGnxoKPeJb02kwQEZ7EZLYO', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{}', name='get_current_time'), tool_calls=None))], created=1752068355, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=76, total_tokens=87, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [8]:
# 判斷是否有 function_call
message = response.choices[0].message
print(message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{}', name='get_current_time'), tool_calls=None)


In [ ]:
if message.function_call:
    print("\n=== GPT 決定呼叫的 Function ===")
    print(f"名稱: {message.function_call.name}")
    print(f"參數: {message.function_call.arguments}")

    # 執行本地對應函數
    if message.function_call.name == "get_current_time":
        def get_current_time():
            now = datetime.now()
            print("[本機] 執行 get_current_time()")
            return now.strftime("%Y-%m-%d %H:%M:%S")

        function_result = get_current_time()

        # 將結果回傳給GPT 讓GPT整理成最終回答
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是一個可以呼叫工具的助理。"},
                {"role": "user", "content": user_question},
                message,  # 原本GPT回傳欲呼叫函數的訊息
                {
                    "role": "function",
                    "name": "get_current_time",
                    "content": function_result
                }
            ]
        )

        # 印出 GPT 最終整理的回答
        final_message = second_response.choices[0].message.content
        print("\n=== GPT 最終回答 ===")
        print(final_message)
else:
    # GPT 沒有呼叫函數，直接回應
    print("\n=== GPT 回覆（未呼叫函數） ===")
    print(message.content)


=== GPT 決定呼叫的 Function ===
名稱: get_current_time
參數: {}
[本機] 執行 get_current_time()

=== GPT 最終回答 ===
現在的時間是2025年7月9日 21:39。
